In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

boston_data = pd.read_csv("BostonHousing.csv")
print(boston_data.head())

X = boston_data.drop(columns=["medv"])
y = boston_data["medv"]

correlation_coeffs = np.abs(boston_data.corr()["medv"]).drop("medv")
best_feature_name = correlation_coeffs.idxmax()

print(f"The attribute that best follows the linear relationship with the output price is: {best_feature_name}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_with_bias = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]

analytic_solution = np.linalg.inv(X_with_bias.T.dot(X_with_bias)).dot(X_with_bias.T).dot(y)

print("Analytic solution coefficients:", analytic_solution)

class LinearRegressionGradientDescent():
    def _init_(self, lr=0.0001, max_iter=10000, tolerance=1e-6):
        self.lr = lr
        self.max_iter = max_iter
        self.tolerance = tolerance

    def fit(self, X, y):
        X_with_bias = np.c_[np.ones((X.shape[0], 1)), X]
        self.weights = np.zeros(X_with_bias.shape[1])

        for _ in range(self.max_iter):
            gradients = -2 * X_with_bias.T.dot(y - X_with_bias.dot(self.weights))
            if np.all(np.abs(gradients) < self.tolerance):
                break
            self.weights -= self.lr * gradients

    def predict(self, X):
        X_with_bias = np.c_[np.ones((X.shape[0], 1)), X]
        return X_with_bias.dot(self.weights)

lr_full_batch_scaled = LinearRegressionGradientDescent()
lr_full_batch_scaled.fit(X_scaled, y)
print("Gradient descent (Full-batch) coefficients (scaled features):", lr_full_batch_scaled.weights)

class LinearRegressionStochasticGradientDescent():
    def _init_(self, lr=0.0001, max_iter=1000, tolerance=1e-6, batch_size=1):
        self.lr = lr
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.batch_size = batch_size

    def fit(self, X, y):
        X_with_bias = np.c_[np.ones((X.shape[0], 1)), X]
        self.weights = np.zeros(X_with_bias.shape[1])

        for _ in range(self.max_iter):
            indices = np.random.permutation(X_with_bias.shape[0])
            for start_idx in range(0, X_with_bias.shape[0], self.batch_size):
                batch_indices = indices[start_idx:start_idx + self.batch_size]
                X_batch = X_with_bias[batch_indices]
                y_batch = y[batch_indices]
                gradients = -2 * X_batch.T.dot(y_batch - X_batch.dot(self.weights))
                if np.all(np.abs(gradients) < self.tolerance):
                    break
                self.weights -= self.lr * gradients

    def predict(self, X):
        X_with_bias = np.c_[np.ones((X.shape[0], 1)), X]
        return X_with_bias.dot(self.weights)

lr_stochastic_scaled = LinearRegressionStochasticGradientDescent(lr=0.0001, max_iter=10000, batch_size=1)
lr_stochastic_scaled.fit(X_scaled, y)
print("Gradient descent (Stochastic) coefficients (scaled features):", lr_stochastic_scaled.weights)

      crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        b  lstat  medv  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2  
The attribute that best follows the linear relationship with the output price is: lstat
Analytic solution coefficients: [ 2.25328063e+01 -9.28146064e-01  1.08156863e+00  1.40899997e-01
  6.81739725e-01 -2.05671827e+00  2.67423017e+00  1.94660717e-02
 -3.10404426e+00  2.66221764e+00 -2.07678168e+00 -2.06060666e+00
  8.49268418e-01 -3.74362713e+00]


AttributeError: 'LinearRegressionGradientDescent' object has no attribute 'max_iter'